___
### <center> CO2 Emission estimation & equivalent in : Hectares of forest, flights and train trajectories</center>
___

This notebook uses a basic interactive display to show the CO2 impact equivalent of any technology working a certain amount of time (between a start and end date) with a certain power (in kW).

After specifying the values above, the display shows :
- The number of hectares required to compensate the emitted C02 in the same period.
- The number of flights (between Paris and Shanghai) to reach the same amount of CO2.
- The number of train travels (between Paris and Lyon) to reach the same amount of CO2.

This simulator takes into consideration the change of the *gCO2/kWh* rate through time (specificly in France) by using the public data found [HERE](https://odre.opendatasoft.com/explore/dataset/eco2mix-national-tr/information/?disjunctive.nature&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJsaW5lIiwiZnVuYyI6IlNVTSIsInlBeGlzIjoidGF1eF9jbzIiLCJjb2xvciI6IiNlYTUyNTQiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZX1dLCJ4QXhpcyI6ImRhdGVfaGV1cmUiLCJtYXhwb2ludHMiOjIwMCwidGltZXNjYWxlIjoibWludXRlIiwic29ydCI6IiIsImNvbmZpZyI6eyJkYXRhc2V0IjoiZWNvMm1peC1uYXRpb25hbC10ciIsIm9wdGlvbnMiOnsiZGlzanVuY3RpdmUubmF0dXJlIjp0cnVlfX19XSwidGltZXNjYWxlIjoiIiwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D).

N.B : The simulation will appear at the end of the notebook after running it.
___

In [ ]:
# installing the necessary dependencies
!sudo python3.8 -m pip install --upgrade pandas plotly ipywidgets folium geopandas

In [103]:
# some useful imports
from datetime import datetime
from math import pi, sqrt

import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets

In [104]:
# defining the estimation functions
def get_num_hectares(start_date, end_date, co2_t):
    co2_hect_day = 11.8 / 365
    num_days = max(abs(end_date-start_date).days, 1)
    return co2_t / (co2_hect_day * num_days )

def get_num_flights_CDG_SHANG(co2_t):
    flight_tco2 = 1.4
    print(co2_t)
    return int(co2_t / flight_tco2)

def get_num_tgv_Paris_Lyon(co2_t):
    tgv_tco2 = 0.76 / 1e03
    return int(co2_t / tgv_tco2)

In [105]:
# setting default values
grenoble_coord = {'lon':[5.72326476215146], 'lat':[45.178485890358424]}

# preparing the data
df = pd.read_csv('data/taux_co2.csv')
df['Date - Heure'] = pd.to_datetime(df['Date - Heure'])
df.sort_values(by='Date - Heure', inplace=True)

In [106]:
# Preparing graph traces
trace1 = go.Scatter(x=df['Date - Heure'], y=(df['Taux de CO2 (g/kWh)'].cumsum() / 4) / 1e06,
                        mode='lines', # 'lines' or 'markers'
                        name='emission', line_color='green')
trace2 = go.Scatter(x=df['Date - Heure'], y=df['Taux de CO2 (g/kWh)'],
                        mode='lines', # 'lines' or 'markers'
                        name='taux', yaxis='y2', line_color='red')
trace_map = go.Scattermapbox(fillcolor='firebrick',
                        lat=['45.178485890358424'],
                        lon=['5.72326476215146'],
                        mode='markers')

In [107]:

try:
    # setting the default map center
    coord_df = pd.DataFrame(grenoble_coord)
    gdf = gpd.GeoDataFrame(coord_df, geometry=gpd.points_from_xy(coord_df.lon, coord_df.lat))

    # prep geometry and layouts
    map_layout = go.Layout(title="Compensation par surface de foret (Ha)")
    map = go.FigureWidget(data=trace_map, layout=map_layout)

    graph_layout = go.Layout(title="Estimation de l'emission CO2 et son equivalent en Hectares par puissance",
                        yaxis=dict(title='emission CO2 (TCO2)'),
                        yaxis2=dict(title='taux CO2 (gCO2/kWh)',
                        overlaying='y',
                        side='right'))
    graph = go.FigureWidget(data=[trace1, trace2], layout=graph_layout)


    # updating layouts:
    map.update_layout(
        mapbox = {
            'style': "open-street-map",
            'center': {'lon': 5.72326476215146, 'lat': 45.178485890358424},
            'zoom': 10},
        showlegend = False)
    map.update_layout(width=500)

    graph.update_layout(width=1000)
    graph.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    # generating the first area (in m)
    cgeo = (
        gdf.set_crs("epsg:4326")
        .sample(1)
        .pipe(lambda d: d.to_crs(d.estimate_utm_crs()))["geometry"]
        .centroid.buffer(1000)
        .to_crs("epsg:4326")
        .__geo_interface__
    )

    # add circle geometry as layer to mapbox figure
    map.update_layout(
        mapbox={
            "layers": [
                {"source": cgeo, "color": "PaleGreen", "type": "fill", "opacity":.5},
                {"source": cgeo, "color": "black", "type": "line", "opacity": 0.1},
            ]
        }
    )
except:
    None

In [108]:
# setting the dynamic widgets
power_value = widgets.FloatText(
            description='Power (kW):',
            disabled=False
        )

start_date = widgets.DatePicker(
            value = min(df['Date - Heure']),
            description='Start Date',
            disabled=False
        )

end_date = widgets.DatePicker(
            value = max(df['Date - Heure']),
            description='End Date',
            disabled=False
        )

In [109]:
# setting response widgets
p_file = open("plane.png", "rb")
img_plane = widgets.Image(
            value=p_file.read(),
            format='png',
            width=40,
            height=40
        )
par_shang_est = widgets.FloatText(
            description='(PAR-SHANG)',
            disabled=False
        )

t_file = open("train.png", "rb")
img_train = widgets.Image(
            value=t_file.read(),
            format='png',
            width=40,
            height=40
        )
tgv_par_lys_est = widgets.FloatText(
            description='(Lyon-Paris)',
            disabled=False
        )

p_file.close()
t_file.close()

In [110]:
# update functions
def validate():
    if power_value.value  > 0:
        return True
    else:
        return False

def response(change):
    if validate():
        # filter by date
        tmp_df = df[(df['Date - Heure'] >= str(start_date.value)) & (df['Date - Heure'] <= str(end_date.value))]

        # update the estimation values:
        co2_t = max(tmp_df['Taux de CO2 (g/kWh)'].cumsum() * power_value.value / 4) / 1e06
        s_h = get_num_hectares(start_date.value, end_date.value, co2_t)
        r_m2 = sqrt((s_h * 1e04) / pi)

        # getting the train/plane equivalents
        num_train = get_num_tgv_Paris_Lyon(co2_t)
        num_plane = get_num_flights_CDG_SHANG(co2_t)
        tgv_par_lys_est.value = num_train
        par_shang_est.value = num_plane

        # update the graph
        with graph.batch_update():
            graph.data[0].x = tmp_df['Date - Heure']
            graph.data[1].x = tmp_df['Date - Heure']
            graph.data[0].y = (tmp_df['Taux de CO2 (g/kWh)'].cumsum() * power_value.value / 4) / 1e06
            graph.data[1].y = tmp_df['Taux de CO2 (g/kWh)']

        # update the map
        with map.batch_update():
            ncgeo = (
                gdf.set_crs("epsg:4326")
                .sample(1)
                .pipe(lambda d: d.to_crs(d.estimate_utm_crs()))["geometry"]
                .centroid.buffer(r_m2)
                .to_crs("epsg:4326")
                .__geo_interface__
            )
            map.update_layout(
                mapbox={
                    "layers": [
                        {"source": ncgeo, "color": "PaleGreen", "type": "fill", "opacity":.5},
                        {"source": ncgeo, "color": "black", "type": "line", "opacity": 0.1},
                    ]
                }
            )
            map.data[0].text = f"{int(s_h)} Hectares of forest needed"

# opserve if any change occurs
power_value.observe(response, names="value")
start_date.observe(response, names="value")
end_date.observe(response, names="value")

In [111]:
# setting containers and displaying the data
container1 = widgets.HBox([power_value, start_date, end_date])
container2 = widgets.HBox([img_plane, par_shang_est, img_train, tgv_par_lys_est])
container3 = widgets.HBox([graph, map])
widgets.VBox([container1, container2, container3])